In [1]:
import numpy as np
import pandas as pd
from tempfile import TemporaryFile
from PIL import Image
import glob
import matplotlib.pyplot as plt 
# ---------------------------
# ------ MODELING -----------
from keras.preprocessing.image import array_to_img, \
    img_to_array, load_img
    
    
    
%matplotlib inline

Using TensorFlow backend.
/Users/dad/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# didn't save dictionary of wood names
# rather than passing it 
# I'll recreate here
def get_file_list(location = '../images/*.*'):
    return sorted(glob.glob(location))
# files in /images folder
filelist_images = get_file_list()

# save filelist_images to done.csv

done_files = np.array(filelist_images)
np.savetxt('../images/done/done.csv', 
           done_files, 
           delimiter=",",
           fmt='%s')

# open done.csv file and save as done list
completed = pd.read_csv('../images/done/done.csv', header = None)
is_done = np.array(completed)
done = []
length = len(is_done)
for i in range(length):
    done.append(is_done[i][0])
    
# Verifies that no new files to add
filelist_images == done
# ------------ FIXED FOR ALL FILES -------------
size = 180 # size of image
border = 5 # remove any weirdness at egde

# Create loop through each file
total_rows = 0
file_indexing = [[]]
images_dict = {}
for file in filelist_images:
    image = Image.open(file)
    array = np.array(image)
    
    # array in now grey scale
    height = array.shape[0]
    width = array.shape[1]
    steps_h = int((height-2*border)/(size/2))
    steps_w = int((width-2*border)/(size/2))
    num_slices = (steps_w-1) * (steps_h-1)
    num_rows = num_slices * 8
    file.split('ges/')[1]
    stuff = [file.split('ges/')[1], file, num_rows, file.split('ges/')[1].split('_v')[0]]
    file_indexing.append(stuff)
    total_rows += num_rows
   
if [] in file_indexing:
    file_indexing.remove([])
    
print('total number of rows: ', total_rows)
print('2nd row of file indexing \n', file_indexing[1])

# get number of catagories and 
# list of woods being looked at
names = []
for row in range(len(file_indexing)):
    names.append(file_indexing[row][3])
unique_names = set(names)
unique_names = list(unique_names)
wood_names_map = {}

for i in range(len(unique_names)):
    print(unique_names[i])
    wood_names_map[unique_names[i]] = i

# print(wood_names_map)
wood_index_map = {v: k for k, v in wood_names_map.items()}
# print(wood_index_map)
# for line in file_indexing:
#     print(file_indexing[i])



total number of rows:  98128
2nd row of file indexing 
 ['beech_v02.jpg', '../images/beech_v02.jpg', 560, 'beech']
white_oak
maple
red_oak
walnut
beech
cherry


In [3]:
# Open Files
X = np.load('../data/X.npy')
y = np.load('../data/y.npy')

In [4]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
tts = train_test_split
X_train, X_test, y_train, y_test = tts(
    X, y, test_size = .2)

In [5]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
cm = confusion_matrix
cr = classification_report

In [6]:
# Check to see what they look like
print("X_train, and y_train", X_train.shape, y_train.shape)
print("X_test, and y_test", X_test.shape, y_test.shape)

X_train, and y_train (78502, 180, 180) (78502, 1)
X_test, and y_test (19626, 180, 180) (19626, 1)


In [7]:
len(unique_names)

6

In [8]:
nb_classes = len(unique_names)
# convert class vectors to binary class matrices (don't change)
Y_train = np_utils.to_categorical(y_train, nb_classes) # cool
Y_test = np_utils.to_categorical(y_test, nb_classes)   # cool * 2
# They should look different now
print("X_train, and Y_train", X_train.shape, Y_train.shape)
print("X_test, and Y_test", X_test.shape, Y_test.shape)

X_train, and Y_train (78502, 180, 180) (78502, 6)
X_test, and Y_test (19626, 180, 180) (19626, 6)


['white_oak', 'maple', 'red_oak', 'walnut', 'beech', 'cherry']
180


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D

from keras import backend as K

In [11]:
batch_size = 50 # number of training samples used at a time to update the weights
nb_epoch = 4    # number of passes
# input image dimensions
img_rows, img_cols = size, size 
input_shape = (img_rows, img_cols, 1)
# number of convolutional filters to use
nb_filters = 8
# size of pooling area for max pooling
pool_size = (2, 2) # decreases image size, and helps to avoid overfitting
# convolution kernel size
kernel_size = (4, 4) # slides over image to learn features
# reshape image for Keras, note that image_dim_ordering set in ~.keras/keras.json
# don't change any of this
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [12]:
# don't change conversion or normalization
X_train = X_train.astype('float32') # data was uint8 [0-255]
X_test = X_test.astype('float32')  # data was uint8 [0-255]
X_train /= 255 # normalizing (scaling from 0 to 1)
X_test /= 255  # normalizing (scaling from 0 to 1)
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (78502, 180, 180, 1)
78502 train samples
19626 test samples


In [28]:
model = Sequential() # model is a linear stack of layers (don't change)

In [29]:
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                    padding='valid',
                    input_shape=input_shape)) #first conv. layer (keep layer)
model.add(Activation('tanh')) # Activation specification necessary for Conv2D and Dense layers
print('Original Activation Level hyperbolic tan', )
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]))) #2nd conv. layer (keep layer)
model.add(Activation('tanh'))
print('Model after first Convulution ', model.output_shape)


model.add(MaxPooling2D(pool_size=pool_size)) # decreases size, helps prevent overfitting
model.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting
print('Model after first MaxPooling ', model.output_shape)




model.add(Flatten()) # necessary to flatten before going into conventional dense layer (keep layer)
print('Model flattened out to ', model.output_shape)

# now start a typical neural network
model.add(Dense(32)) # (only) 32 neurons in this layer, really?  (keep layer)
model.add(Activation('relu'))
print('First Dense Layer 32 ', model.output_shape)

model.add(Dense(64)) # (only) 32 neurons in this layer, really?  (keep layer)
model.add(Activation('relu'))
print('2nd Dense Layer, relu ', model.output_shape)



model.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting

model.add(Dense(nb_classes)) # 10 final nodes (one for each class) (keep layer)
model.add(Activation('softmax')) # keep softmax at end to pick between classes 0-9
print('2nd Dense Layer - number of classes, softmax ', model.output_shape)

# many optimizers available
# see https://keras.io/optimizers/#usage-of-optimizers
# suggest you keep loss at 'categorical_crossentropy' for this multiclass problem,
# and metrics at 'accuracy'
# suggest limiting optimizers to one of these: 'adam', 'adadelta', 'sgd'
# how are we going to solve and evaluate it:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])





Original Activation Level hyperbolic tan
Model after first Convulution  (None, 174, 174, 8)
Model after first MaxPooling  (None, 87, 87, 8)
Model flattened out to  (None, 60552)
First Dense Layer 32  (None, 32)
2nd Dense Layer, relu  (None, 64)
2nd Dense Layer - number of classes, softmax  (None, 6)


In [30]:
# during fit process watch train and test error simultaneously
cnn_fit = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
cnn_pred = model.predict(X_test)

Train on 78502 samples, validate on 19626 samples
Epoch 1/4
78502/78502 [==============================] - 3097s 39ms/step - loss: 1.0606 - acc: 0.6141 - val_loss: 0.9541 - val_acc: 0.6500
Epoch 2/4
78502/78502 [==============================] - 3097s 39ms/step - loss: 0.6745 - acc: 0.7575 - val_loss: 0.5395 - val_acc: 0.7902
Epoch 3/4
78502/78502 [==============================] - 3090s 39ms/step - loss: 0.5211 - acc: 0.8092 - val_loss: 0.4287 - val_acc: 0.8339
Epoch 4/4
78502/78502 [==============================] - 3103s 40ms/step - loss: 0.4076 - acc: 0.8539 - val_loss: 0.7508 - val_acc: 0.7458


In [31]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1]) # this is the one we care about

Test score: 0.75083406038
Test accuracy: 0.745847345346


In [32]:
model.save(filepath = 'cnn_model')

In [33]:
model.layers

In [34]:
cnn_pred_1 = cnn_pred.argmax(axis=-1)

In [41]:
print('classification report \n')
print(cr(y_test, cnn_pred_1))
print('confusion matrix\n')
print(cm(y_test, cnn_pred_1))
for i in range(len(wood_index_map)):
    print(i,'--', wood_index_map[i])


print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])


classification report 

             precision    recall  f1-score   support

        0.0       0.82      0.94      0.87      6706
        1.0       0.51      0.52      0.52      1230
        2.0       0.00      0.00      0.00       148
        3.0       0.54      0.87      0.66      2721
        4.0       0.99      0.63      0.77      6621
        5.0       0.57      0.53      0.55      2200

avg / total       0.78      0.75      0.74     19626

confusion matrix

[[6271    0   17  264   42  112]
 [   0  645    0  498    1   86]
 [   5   20    0   17    5  101]
 [ 182   38    0 2359    2  140]
 [1195  431    0  351 4203  441]
 [  16  124    2  898    0 1160]]
0 -- white_oak
1 -- maple
2 -- red_oak
3 -- walnut
4 -- beech
5 -- cherry


Test score: 0.75083406038
Test accuracy: 0.745847345346


In [39]:
print(wood_index_map[0])

white_oak


In [51]:
Y_train.sum(axis=0)

array([ 26974.,   4946.,    548.,  10895.,  26179.,   8960.])

In [48]:
Y_test.shape

(19626, 6)

In [49]:
Y_test[:4]

array([[ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.]])

In [54]:
model_4 = Sequential()
model_4.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                    padding='valid',
                    input_shape=input_shape)) #first conv. layer (keep layer)
model_4.add(Activation('tanh')) # Activation specification necessary for Conv2D and Dense layers
print('Original Activation Level hyperbolic tan', )
model_4.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]))) #2nd conv. layer (keep layer)
model_4.add(Activation('tanh'))
print('Model after first Convulution ', model_4.output_shape)


model_4.add(MaxPooling2D(pool_size=pool_size)) # decreases size, helps prevent overfitting
model_4.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting
print('Model after first MaxPooling ', model_4.output_shape)




model_4.add(Flatten()) # necessary to flatten before going into conventional dense layer (keep layer)
print('Model flattened out to ', model_4.output_shape)

# now start a typical neural network
model_4.add(Dense(32)) # (only) 32 neurons in this layer, really?  (keep layer)
model_4.add(Activation('relu'))
print('First Dense Layer 32 ', model_4.output_shape)

model_4.add(Dense(64)) # (only) 32 neurons in this layer, really?  (keep layer)
model_4.add(Activation('relu'))
print('2nd Dense Layer, relu ', model_4.output_shape)

model_4.add(Dense(128)) # (only) 32 neurons in this layer, really?  (keep layer)
model_4.add(Activation('relu'))
print('3nd Dense Layer, relu ', model_4.output_shape)


model_4.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting

model_4.add(Dense(nb_classes)) # 10 final nodes (one for each class) (keep layer)
model_4.add(Activation('softmax')) # keep softmax at end to pick between classes 0-9
print('Last Dense Layer - number of classes, softmax ', model_4.output_shape)

model_4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Original Activation Level hyperbolic tan
Model after first Convulution  (None, 174, 174, 8)
Model after first MaxPooling  (None, 87, 87, 8)
Model flattened out to  (None, 60552)
First Dense Layer 32  (None, 32)
2nd Dense Layer, relu  (None, 64)
3nd Dense Layer, relu  (None, 128)
Last Dense Layer - number of classes, softmax  (None, 6)


In [55]:
# during fit process watch train and test error simultaneously
cnn_fit_4 = model_4.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
cnn_pred_4 = model_.predict(X_test)
# -------------------------------------
score_4 = model_4.evaluate(X_test, Y_test, verbose=0)
# -------------------------------------
model_4.save(filepath = 'cnn_model_4')

print(model_4.layers)

cnn_pred_4_1 = cnn_pred_4.argmax(axis=-1)
print('classification report \n')
print(cr(y_test, cnn_pred_4_1))
print('confusion matrix\n')
print(cm(y_test, cnn_pred_4_1))
for i in range(len(wood_index_map)):
    print(i,'--', wood_index_map[i])


print('\n')
print('Test score:', score_4[0])
print('Test accuracy:', score_4[1])



Train on 78502 samples, validate on 19626 samples
Epoch 1/4
78502/78502 [==============================] - 3099s 39ms/step - loss: 0.9744 - acc: 0.6429 - val_loss: 0.7066 - val_acc: 0.7444
Epoch 2/4
78502/78502 [==============================] - 3016s 38ms/step - loss: 0.5851 - acc: 0.7807 - val_loss: 0.4713 - val_acc: 0.8236
Epoch 3/4
78502/78502 [==============================] - 3070s 39ms/step - loss: 0.4319 - acc: 0.8339 - val_loss: 0.4149 - val_acc: 0.8439
Epoch 4/4
78502/78502 [==============================] - 3147s 40ms/step - loss: 0.3622 - acc: 0.8587 - val_loss: 0.4391 - val_acc: 0.8315


NameError: name 'model_' is not defined

In [56]:
cnn_pred_4 = model_4.predict(X_test)
# -------------------------------------
score_4 = model_4.evaluate(X_test, Y_test, verbose=0)
# -------------------------------------
model_4.save(filepath = 'cnn_model_4')

print(model_4.layers)

cnn_pred_4_1 = cnn_pred_4.argmax(axis=-1)
print('classification report \n')
print(cr(y_test, cnn_pred_4_1))
print('confusion matrix\n')
print(cm(y_test, cnn_pred_4_1))
for i in range(len(wood_index_map)):
    print(i,'--', wood_index_map[i])


print('\n')
print('Test score:', score_4[0])
print('Test accuracy:', score_4[1])



[<keras.layers.convolutional.Conv2D object at 0x20a685c0f0>, <keras.layers.core.Activation object at 0x20a685c1d0>, <keras.layers.convolutional.Conv2D object at 0x20a685c4e0>, <keras.layers.core.Activation object at 0x20a685ce10>, <keras.layers.pooling.MaxPooling2D object at 0x20a72c3dd8>, <keras.layers.core.Dropout object at 0x20a6befe80>, <keras.layers.core.Flatten object at 0x20a68699e8>, <keras.layers.core.Dense object at 0x20a6c01898>, <keras.layers.core.Activation object at 0x20a6bc81d0>, <keras.layers.core.Dense object at 0x20a6bd2c50>, <keras.layers.core.Activation object at 0x20a6bb4da0>, <keras.layers.core.Dense object at 0x20a6b8a6a0>, <keras.layers.core.Activation object at 0x20a6b920b8>, <keras.layers.core.Dropout object at 0x20a6b5e278>, <keras.layers.core.Dense object at 0x20a6b8a208>, <keras.layers.core.Activation object at 0x20a6b75160>]
classification report 

             precision    recall  f1-score   support

        0.0       0.95      0.90      0.93      6706
  

/Users/dad/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [58]:
print('classification report \n')
print(cr(y_test, cnn_pred_4_1))
print('confusion matrix\n')
print(cm(y_test, cnn_pred_4_1))
for i in range(len(wood_index_map)):
    print(i,'--', wood_index_map[i])


print('\n')

print('Test accuracy:', score_4[1])



classification report 

             precision    recall  f1-score   support

        0.0       0.95      0.90      0.93      6706
        1.0       0.64      0.83      0.72      1230
        2.0       0.00      0.00      0.00       148
        3.0       0.72      0.57      0.64      2721
        4.0       0.85      0.97      0.91      6621
        5.0       0.65      0.57      0.61      2200

avg / total       0.82      0.83      0.82     19626

confusion matrix

[[6060    2    0   63  540   41]
 [   0 1024    0    8   92  106]
 [  14    4    0   17   21   92]
 [ 234  171    0 1551  335  430]
 [  26  136    0    9 6429   21]
 [  36  265    0  494  150 1255]]
0 -- white_oak
1 -- maple
2 -- red_oak
3 -- walnut
4 -- beech
5 -- cherry


Test accuracy: 0.831499031896


/Users/dad/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [59]:
model_4.save(filepath = 'cnn_model_5')

In [63]:
batch_size = 80 # number of training samples used at a time to update the weights
nb_epoch = 5    # number of passes
# input image dimensions
img_rows, img_cols = size, size 
input_shape = (img_rows, img_cols, 1)
# number of convolutional filters to use
nb_filters = 12
# size of pooling area for max pooling
pool_size = (2, 2) 
# decreases image size, and helps to avoid overfitting
# convolution kernel size
kernel_size = (8, 8) # slides over image to learn features
# reshape image for Keras, note that image_dim_ordering set in ~.keras/keras.json
# don't change any of this

model = Sequential()
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                    padding='valid',
                    input_shape=input_shape)) 
#first conv. layer (keep layer)
model.add(Activation('tanh')) # Activation specification necessary for Conv2D and Dense layers
print('Original Activation Level hyperbolic tan', )
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]))) #2nd conv. layer (keep layer)
model.add(Activation('tanh'))
print('Model after first Convulution ', model.output_shape)

# 2nd conv layer
model.add(Activation('relu')) # Activation specification necessary for Conv2D and Dense layers
print('2nd Convulution Level relu', )
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))
print('Model after second Convulution ', model.output_shape)


model.add(MaxPooling2D(pool_size=pool_size)) # decreases size, helps prevent overfitting
model.add(Dropout(0.2)) # zeros out some fraction of inputs, helps prevent overfitting
print('Model after first MaxPooling ', model.output_shape)




model.add(Flatten()) # necessary to flatten before going into conventional dense layer (keep layer)
print('Model flattened out to ', model.output_shape)

# now start a typical neural network
model.add(Dense(32)) # (only) 32 neurons in this layer, really?  (keep layer)
model.add(Activation('relu'))
print('First Dense Layer 32 ', model.output_shape)

model.add(Dense(64)) # (only) 32 neurons in this layer, really?  (keep layer)
model.add(Activation('relu'))
print('2nd Dense Layer, relu ', model.output_shape)

model.add(Dense(128)) # (only) 32 neurons in this layer, really?  (keep layer)
model.add(Activation('relu'))
print('3nd Dense Layer, relu ', model.output_shape)


model.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting

model.add(Dense(nb_classes)) # 10 final nodes (one for each class) (keep layer)
model.add(Activation('softmax')) # keep softmax at end to pick between classes 0-9
print('Last Dense Layer - number of classes, softmax ', model_4.output_shape)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Original Activation Level hyperbolic tan
Model after first Convulution  (None, 166, 166, 12)
2nd Convulution Level relu
Model after second Convulution  (None, 159, 159, 12)
Model after first MaxPooling  (None, 79, 79, 12)
Model flattened out to  (None, 74892)
First Dense Layer 32  (None, 32)
2nd Dense Layer, relu  (None, 64)
3nd Dense Layer, relu  (None, 128)
Last Dense Layer - number of classes, softmax  (None, 6)


In [ ]:
# during fit process watch train and test error simultaneously
cnn_fit_5 = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
cnn_pred_5 = model.predict(X_test)
# -------------------------------------
score_5 = model_4.evaluate(X_test, Y_test, verbose=0)
# -------------------------------------
model.save(filepath = 'cnn_model_5')

print(model.layers)

cnn_pred_5_1 = cnn_pred_5.argmax(axis=-1)
print('classification report \n')
print(cr(y_test, cnn_pred_5_1))
print('confusion matrix\n')
print(cm(y_test, cnn_pred_5_1))
for i in range(len(wood_index_map)):
    print(i,'--', wood_index_map[i])


print('\n')
print('Test score:', score_5[0])
print('Test accuracy:', score_5[1])




Train on 78502 samples, validate on 19626 samples
Epoch 1/5
65280/78502 [=======================>......] - ETA: 1:28:43 - loss: 1.4235 - acc: 0.3600